In [21]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime
from datetime import timedelta

# Functions

In [22]:
# Function to load data to Mysql
def load_MySQL(table_name,new_dataframe):

    # Replace the following with your own database connection details
    db_username = 'u134043424_user'
    db_password = 'l4K#SF26Y=Q'
    db_host = '45.152.46.52'
    db_port = '3306'
    db_name = 'u134043424_learning_db'

    # Create the database engine
    engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

    # Load the DataFrame into MySQL table
    new_dataframe.to_sql(name=table_name, con=engine, if_exists='replace', index=False, chunksize=100_000)

    print("DataFrame has been loaded into the database.")

    return None

In [23]:
# Function to get data from database
def get_data(sql_query):
    
# Replace these variables with your actual database connection details
    db_username = 'u134043424_user'
    db_password = 'l4K#SF26Y=Q'
    db_host = '45.152.46.52'
    db_port = '3306'
    db_name = 'u134043424_learning_db'

    # Create the database engine
    engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

    # Read the table into a pandas DataFrame
    df = pd.read_sql_query(sql_query, con=engine)

    return df

In [24]:
# Function to create simulation
# The parameters of the function can be change so it can semulate diferent values
def create_simulation(date_start, # Its the date when the sumulation start
                      date_finish, # Its the date when the sumulation finish
                      mu = 1, # Its the middle value that the distribution its going to take
                      sigma = 1, # Its the variance of the simulation
                      min = 0, # Its the min value that the simulation can take
                      max = 2): # Its the max value that the simulation can take

    # Create date range
    range_dates = pd.date_range(start=date_start, end=date_finish, freq='S')

    # Create DataFrame
    df_dates = pd.DataFrame(range_dates, columns=['timestamp'])

    # Generate the random normal distribution
    random_num = np.random.normal(mu, sigma, size=len(df_dates))

    # Add column to DataFrame
    df_dates['value'] = random_num

    df_dates['value'] = np.where(df_dates['value'] < min, min, df_dates['value'])

    df_dates['value'] = np.where(df_dates['value'] > max, max, df_dates['value'])
    
    return df_dates


In [25]:
# Funcrion to adjust dataframe to XML data structure
def complete_df(df1, df2, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence):
    dft = pd.concat([df1, df2])
    dft['deviceName'] = deviceName
    dft['deviceUuid'] = deviceUuid
    dft['component'] = component
    dft['componentName'] = componentName
    dft['componentId'] = componentId
    dft['measurementType'] = measurementType
    dft['dataItemId'] = dataItemId
    dft['name'] = name
    dft['sequence'] = sequence
    
    dft = dft[['deviceName', 'deviceUuid', 'component', 'componentName', 'componentId', 'measurementType', 'dataItemId', 'timestamp', 'name', 'sequence', 'subType', 'value', 'mantenance']]
    return dft

# Read Data

In [26]:
root = ET.parse('StratasysSample.xml')

# Prepare a list to hold extracted data
data = []

# Iterate through each DeviceStream and its ComponentStreams
for device_stream in root.findall('.//{urn:mtconnect.org:MTConnectStreams:1.5}DeviceStream'):
    device_name = device_stream.attrib.get('name')
    device_uuid = device_stream.attrib.get('uuid')
    for component_stream in device_stream.findall('.//{urn:mtconnect.org:MTConnectStreams:1.5}ComponentStream'):
        component = component_stream.attrib.get('component')
        component_name = component_stream.attrib.get('name')
        component_id = component_stream.attrib.get('componentId')
        # Extract samples
        for samples in component_stream.findall('.//{urn:mtconnect.org:MTConnectStreams:1.5}Samples'):
            for sample in samples:
                # Extract all attributes of the sample
                sample_data = {attr: sample.attrib.get(attr) for attr in sample.attrib}
                sample_data['value'] = sample.text
                sample_data['componentName'] = component_name
                sample_data['componentId'] = component_id
                sample_data['deviceName'] = device_name
                sample_data['deviceUuid'] = device_uuid
                sample_data['component'] = component
                sample_data['measurementType'] = sample.tag.split('}')[1]  # To get 'Amperage', 'Voltage', etc.
                data.append(sample_data)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)

df = df[['deviceName','deviceUuid','component','componentName','componentId','measurementType','dataItemId', 'timestamp', 'name', 'sequence', 'subType', 'value']].copy()

## Create Tables

In [27]:
# exploration of the table
df_amperage_enclosure = df[(df['measurementType'] == 'Amperage')&(df['component'] == 'Enclosure')]
df_amperage_linear = df[(df['measurementType'] == 'Amperage')&(df['component'] == 'Linear')]
df_voltage = df[df['measurementType'] == 'Voltage']
df_temperature = df[df['measurementType'] == 'Temperature']
df_rotaryvelocity = df[df['measurementType'] == 'RotaryVelocity']
df_powerfactor = df[df['measurementType'] == 'PowerFactor']

# Simulation

In [28]:
# Parameters of simulation
number_of_days = 1
start_date = (datetime.now()- timedelta(days=number_of_days,hours=0,minutes=0)).strftime('%Y-%m-%d %H:%M:%S')
finish_date = (datetime.now()).strftime('%Y-%m-%d %H:%M:%S')

## Temperature

In [29]:
# Parameters of machine
dataItemId = 'ovenHeater1Com'
name = 'OvenHeater1CommandedTemperature'
measurementType = 'Temperature'
component = 'Enclosure'
componentName = 'Oven'
componentId = 'oven'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'f170'
deviceUuid = 'D12827'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2) 
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final1 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [30]:
# Parameters of machine
dataItemId = 'ovenHeater2Com'
name = 'OvenHeater2CommandedTemperature'
measurementType = 'Temperature'
component = 'Enclosure'
componentName = 'Oven'
componentId = 'oven'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(1/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'f170'
deviceUuid = 'D12827'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final2 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [31]:
# Create temperature dataframe
df_temperature = pd.concat([df_final1, df_final2])

## Speed

In [32]:
# Parameters of machine
dataItemId = 'ovenBlower1Speed'
name = 'OvenBlower1Speed'
measurementType = 'RotaryVelocity'
component = 'Enclosure'
componentName = 'Oven'
componentId = 'oven'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'f170'
deviceUuid = 'D12827'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final1 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [33]:
# Parameters of machine
dataItemId = 'ovenBlower2Speed'
name = 'OvenBlower2Speed'
measurementType = 'RotaryVelocity'
component = 'Enclosure'
componentName = 'Oven'
componentId = 'oven'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(1/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'f170'
deviceUuid = 'D12827'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final2 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [34]:
# create speed dataframe
df_speed = pd.concat([df_final1, df_final2])

## Linear

In [35]:
# Parameters of machine
dataItemId = 'x1Current'
name = 'MotorCurrent'
measurementType = 'Amperage'
component = 'Linear'
componentName = 'X'
componentId = 'x1'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'f170'
deviceUuid = 'D12827'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final1 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [36]:
# Parameters of machine
dataItemId = 'z1Current'
name = 'MotorCurrent'
measurementType = 'Amperage'
component = 'Linear'
componentName = 'Z'
componentId = 'z1'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'f170'
deviceUuid = 'D12827'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final2 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [37]:
# Create linear dataframe
df_linear = pd.concat([df_final1, df_final2])

## Join all tables

In [38]:
# Create final dataframe to load to MySQL
df_final = pd.concat([df_temperature, df_speed, df_linear])

# Load Data

In [39]:
print("Rows to load to MySQL:", len(df_final))
print("Expected time:", (len(df_final)/1_000_000)*1.15, "minutes")

Rows to load to MySQL: 1036812
Expected time: 1.1923337999999999 minutes


In [40]:
# Load the data to MySQL
load_MySQL('Stratasys',df_final)

DataFrame has been loaded into the database.
